<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama2/blob/main/ExemplosGeracaoTexto_Llama2_Langchain_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de Geração de textos usando Llama v2.0 7B 8bit usando Langchain e Transformers by HuggingFace

Exemplo de uso do modelo de linguagem grande Llama v2.0.
- Análise da geração de textos
- Prompts com textos emparelhados
- Injentando padrões no prompt
- Padrão Persona
- Verificação cognitiva
- Pensamento em cadeia
- Refinamento de perguntas

**Toda a execução ocorre no Google Colaboratory.**

Pré-requisitos:
- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.
- Configurar o notebook para usar GPU- Acesse o menu 'Ambiente de Execução -> Alterar o tipo do ambiente de execução -> Acelerador de hardware -> T4 GPU


**Referências**
https://medium.com/the-techlife/using-huggingface-openai-and-cohere-models-with-langchain-db57af14ac5b


**Notebook de referência:**

https://github.com/guardiaum/tutorial-sbbd2023/blob/main/Prompt_Engineering.ipynb


**Lista dos modelos:**

https://huggingface.co/models


**Artigos referências:**

https://dev.to/nithinibhandari1999/how-to-run-llama-2-on-your-local-computer-42g1


**Link biblioteca Huggingface:**

https://github.com/huggingface/transformers




# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Data e hora de execução

In [ ]:
# Biblioteca de date
from datetime import datetime

data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d/%m/%Y %H:%M:%S')

print(data_e_hora_em_texto)

24/04/2024 21:22:08


## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Biblioteca do sistema
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Versão Python

In [ ]:
# Biblioteca do sistema
import sys

print("Versão Python:", sys.version)

Versão Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


## GPU

In [ ]:
# Import de biblioteca
import torch

GPU_ENABLE = torch.cuda.is_available()

if GPU_ENABLE:
    print("GPU está disponível.")
else:
    print("GPU não está disponível.")

GPU está disponível.


## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

Imprime linhas menores.

In [ ]:
def print_linhas_menores(texto, tamanho=120):
  for i in range(0, len(texto), tamanho):
    print(texto[i:i+tamanho])

# 1 - Instalação das bibliotecas

Bibioteca LangChain é um framework de código aberto para o desenvolvimento de aplicações usando modelos de linguagem grandes.

In [ ]:
!pip install langchain==0.1.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


O bitsandbytes é um wrapper leve em torno de funções personalizadas CUDA, em particular otimizadores de 8 bits, multiplicação de matrizes (LLM.int8()) e funções de quantização. É uma dependência do accelerate.

In [ ]:
!pip install bitsandbytes==0.43.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

Accelerate é uma biblioteca que permite que o mesmo código PyTorch seja executado em qualquer configuração distribuída adicionando apenas quatro linhas de código. Otimiza as operações do PyTorch, especialmente na GPU.

https://pypi.org/project/accelerate/

In [ ]:
!pip install accelerate==0.29.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.1 MB/s eta 0:00:00


A Biblioteca A Biblioteca Transformers fornece APIs e ferramentas para baixar e treinar facilmente modelos pré-treinados de última geração para Processamento de linguagem natural, Visão computacional, Áudio, etc.

Fornece uma maneira direta de usar modelos pré-treinados.

In [ ]:
!pip install -U transformers==4.40.0

A Biblioteca huggingface-cli fornece vários comandos para interagir com o Hugging Face Hub a partir da linha de comando. Um desses comandos é o login, que permite aos usuários se autenticarem no Hub usando suas credenciais.

In [ ]:
#!pip install huggingface-hub==0.20.3

## Versão bibliotecas instaladas

In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==0.29.3
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albumentations==1.3.1
altair==4.2.2
annotated-types==0.6.0
anyio==3.7.1
appdirs==1.4.4
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.15.1
astropy==5.3.4
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.14.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.2.0
bitsandbytes==0.43.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.3.4
bqplot==0.12.43
branca==0.7.1
build==1.2.1
CacheControl==0.14.0
cachetools==5.3.3
catalogue==2.0.10
certifi==2024.2.2
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
chex==0.1.86
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
cloudpathlib==0.16.0
cloudpickle==2.2.1
cmake==3.27.9
cmdstanpy==1.2.2
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.4
cons==0.4.6
contextlib2==21.6.0
contourpy==1.2.1
cryptography==42.0.

# 2 - Carregando o LLM



## 2.1 - Login no huggingface

- Lhama 2 não está acessível abertamente e requer solicitação  de acesso. Faça o cadastro no site do https://huggingface.co/join. Depois do login, gere um token de acesso no link https://huggingface.co/settings/tokens.

Insira o token quando solicitado e depois digite Y para adicionar as credenciais.

In [ ]:
# !huggingface-cli login

Se o seu notebook não for público e não desejar incluir o token de acesso toda vez que for executar o notebook preencha o método save_token.


Crie a variável 'HF_TOKEN' com o valor do **Access Token do HuggingFace**. Abra o Google Colab e navegue até a nova seção 'Secrets' na barra lateral e adicione a variável.

In [ ]:
from huggingface_hub.hf_api import HfFolder

if IN_COLAB:

    from google.colab import userdata

    # ACESS_TOKEN = "<valor_do_acess_token"
    ACCESS_TOKEN  = userdata.get('HF_TOKEN')

    HfFolder.save_token(ACCESS_TOKEN)

Mostrando o usuário conectado

In [ ]:
# !huggingface-cli whoami

## 2.2 - Nome do modelo de linguagem

Define o nome do modelo a ser carregado
Lista dos modelos:
  - https://huggingface.co/meta-llama/Llama-2-7b-hf
  - https://huggingface.co/meta-llama/Llama-2-7b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-hf
  - https://huggingface.co/meta-llama/Llama-2-13b-chat-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-hf
  - https://huggingface.co/meta-llama/Llama-2-70b-chat-hf

In [ ]:
#nome_modelo = "meta-llama/Llama-2-7b-hf"
nome_modelo = "meta-llama/Llama-2-7b-chat-hf"

#nome_modelo = "meta-llama/Llama-2-13b-hf"
# nome_modelo = "meta-llama/Llama-2-13b-chat-hf"

# Não roda pois exige GPU A100 e mais espaço em disco
#nome_modelo = "meta-llama/Llama-2-70b-hf"
# nome_modelo = "meta-llama/Llama-2-70b-chat-hf"

## 2.3 - Carrega o tokenizador

Carregando o **tokenizador** da comunidade.

In [ ]:
# Importando as bibliotecas do Tokenizador
from transformers import AutoTokenizer

# Carregando o Tokenizador da comunidade
print('Carregando o tokenizador ' + nome_modelo + ' da comunidade...')

tokenizer = AutoTokenizer.from_pretrained(nome_modelo)

Carregando o tokenizador meta-llama/Llama-2-7b-chat-hf da comunidade...


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Tamanho do vocabulário

In [ ]:
print(len(tokenizer))

32000


## 2.4 - Carregando o LLM

Carregando o **LLM** da comunidade HuggingFace.

Parametrização do from_pretrained
https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu

Carregamento LLama 2 com 4 bits

In [ ]:
# # Importando as bibliotecas do Modelo
# from transformers import BitsAndBytesConfig, AutoModelForCausalLM
# import torch
# import time

# # Guarda o tempo de início do carregamento do modelo
# tempo_inicio = time.time()

# # Carregando o Modelo da comunidade
# print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# # BitsAndBytes é um framework com funções customizadas para
# # otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
# quantization_config = BitsAndBytesConfig(
#    load_in_4bit=True, # Habilita a quantização de 4 bits para comprimir o modelo
#    bnb_4bit_quant_type="nf4", # Define o tipo de dados de quantização nas camadas (`fp4` e `nf4`).
#    bnb_4bit_use_double_quant=True, # Quantização aninhada, onde as constantes de quantização da primeira quantização são quantizadas novamente.
#    bnb_4bit_compute_dtype=torch.bfloat16 # # Os gradientes dos pesos são computados em 16-bit. Define o tipo computacional que pode ser diferente do tempo de entrada. Por exemplo, as entradas podem ser fp32, mas a computação pode ser definida como bf16 para acelerações.
# )

# # Carrega o modelo
# model = AutoModelForCausalLM.from_pretrained(nome_modelo,
#                                              #torch_dtype=torch.float16, #default
#                                              trust_remote_code=True, # Carrega de um repositório confiável
#                                              quantization_config=quantization_config,
#                                              device_map="auto"
#                                              )

# # Coloca o modelo e modo avaliação
# model.eval()

# # Aumentar a velocidade
# # https://huggingface.co/docs/transformers/main/perf_torch_compile
# model = torch.compile(model)

# print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregamento LLama 2 com 8 bits

In [ ]:
# Importando as bibliotecas do Modelo
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
import torch
import time

# Guarda o tempo de início do carregamento do modelo
tempo_inicio = time.time()

# Carregando o Modelo da comunidade
print('Carregando o modelo ' + nome_modelo + ' da comunidade...')

# BitsAndBytes é um framework com funções customizadas para
# otimização com precisão 8-bit, multiplicações de matrizes e funções de quantização
quantization_config = BitsAndBytesConfig(
   load_in_8bit=True, # Habilita a quantização de 8 bits
)

# Carrega o modelo
model = AutoModelForCausalLM.from_pretrained(nome_modelo,
                                             #torch_dtype=torch.float16, #default
                                             trust_remote_code=True, # Carrega de um repositório confiável
                                             quantization_config=quantization_config,
                                             device_map="auto"
                                             )

# Coloca o modelo e modo avaliação
model.eval()

# Aumentar a velocidade
# https://huggingface.co/docs/transformers/main/perf_torch_compile
model = torch.compile(model)

print("Tempo de carregamento do modelo LLM:  {:} (h:mm:ss)".format(formataTempo(time.time() - tempo_inicio)))

Carregando o modelo meta-llama/Llama-2-7b-chat-hf da comunidade...


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Tempo de carregamento do modelo LLM:  0:02:43 (h:mm:ss)


In [ ]:
print(model)

OptimizedModule(
  (_orig_mod): LlamaForCausalLM(
    (model): LlamaModel(
      (embed_tokens): Embedding(32000, 4096)
      (layers): ModuleList(
        (0-31): 32 x LlamaDecoderLayer(
          (self_attn): LlamaSdpaAttention(
            (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (v_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (rotary_emb): LlamaRotaryEmbedding()
          )
          (mlp): LlamaMLP(
            (gate_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (up_proj): Linear8bitLt(in_features=4096, out_features=11008, bias=False)
            (down_proj): Linear8bitLt(in_features=11008, out_features=4096, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): LlamaRMSNorm(

In [ ]:
print(model.config)

LlamaConfig {
  "_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 4096,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": false,
    "_load_in_8bit": true,
    "bnb_4bit_compute_dtype": "float32",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "fp4",
    "bnb_4bit_use_double_quant": false,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": false,
    "load_in_8bit": true,
    "quant_method": "bitsandbytes"
  },
  "rms_norm_eps": 1e-05,
  "rope_

In [ ]:
print(model.config.max_position_embeddings)

4096


Tamanho do vocabulário

In [ ]:
print(model.config.vocab_size)

32000


## 2.5 - Configuração da geração de texto

In [ ]:
# Import das bibliotecas
from transformers import GenerationConfig

# Instância as configurações do modelo
generation_config = GenerationConfig.from_pretrained(nome_modelo)

print("GenerationConfig antes:\n",generation_config)
generation_config.max_new_tokens = 512 # Preenche até um comprimento máximo especificado com o argumento max_length ou até o comprimento de entrada máximo aceitável para o modelo se esse argumento não for fornecido.
#generation_config.max_length = 4096 # (Default 4096)
# Se do_sample é true setar temperature e top_p, caso contrário se do_sample é false remover temperature e top_p.
generation_config.do_sample = True # (Default True) Se definido como True, este parâmetro permite estratégias de decodificação como amostragem multinomial, amostragem multinomial de busca de feixe, amostragem Top-K e amostragem Top-p. Todas essas estratégias selecionam o próximo token da distribuição de probabilidade em todo o vocabulário com vários ajustes específicos da estratégia.
generation_config.temperature = 0.1 # (Default 0.6) A temperatura é um parâmetro que controla a aleatoriedade da saída do LLM. Uma temperatura mais alta resultará em um texto mais criativo e imaginativo, enquanto uma temperatura mais baixa resultará em um texto mais preciso e factual.
#generation_config.top_k = 3  # Top-k diz ao modelo para escolher o próximo token entre os 'k' tokens principais de sua lista, classificados por probabilidade.
#generation_config.top_p = 0.9 # (Default 0.9) Top-p é mais dinâmico que top-k e é frequentemente usado para excluir resultados com probabilidades mais baixas. Portanto, se você definir p como 0,75, excluirá os 25% inferiores dos resultados prováveis.
#generation_config.repetition_penalty = 1.20 # Penaliza a repetição e visa evitar frases que se repetem sem nada de realmente interessante.
#generation_config.num_return_sequences=1, # Retorna uma única sentença da saída.
generation_config.pad_token_id=generation_config.eos_token_id
print("GenerationConfig depois:\n",generation_config)

GenerationConfig antes:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

GenerationConfig depois:
 GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "max_new_tokens": 512,
  "pad_token_id": 2,
  "temperature": 0.1,
  "top_p": 0.9
}



## 2.6 - Cria o pipeline usando Langchain

Cria o pipeline com a classe [HuggingFacePipeline](https://api.python.langchain.com/en/latest/llms/langchain.llms.huggingface_pipeline.HuggingFacePipeline.html) do langchain.

Passagem direta do pipeline Huggingface.

Configura o pipeline do Huggingface usando o modelo e tokenizador previamente carregado e passa para o HuggingFacePipeline do langchain.

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline do HuggingFace
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # return_full_text=True,  # (Default True) Langchain espera o texto completo
    generation_config=generation_config, # Passa as configurações da geração de texto para o pipeline
)

# Carrega o pipeline do Langchain
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(pipeline=pipe)

The model 'OptimizedModule' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus

In [ ]:
print(model_llm)

HuggingFacePipeline
Params: {'model_id': 'gpt2', 'model_kwargs': None, 'pipeline_kwargs': None}


# 3 - Analisando a geração de textos



## 3.1 - Geração de texto simples


In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"
#documento = "How to push elements in a stack"
#documento = "O comando SQL para extrair todos os usuários cujo nome começa com A é:"
#documento = "Bom dia professor, tudo bem ?"
# documento = "The SQL command to extract all the users whose name starts with A is:"
#documento = "How to push elements in a stack"
#documento = "Write code for finding the prime number in python ?"
# documento = "Escrever código para encontrar o número primo em python?"

# Prepara o prompt para enviar ao modelo realizando sua tokenização
# Se pt for especificado, ele retornará tensores em vez de lista de inteiros python e tokenizará os documentos
input = tokenizer(documento, return_tensors="pt")

# Mostra os tokens com seus índices
i = 0
for tup in input.input_ids[0]:
    # print(tup.item())
    print("{} {}".format(i, tokenizer.convert_ids_to_tokens(tup.item())))
    i= i + 1

0 <s>
1 ▁Como
2 ▁emp
3 il
4 har
5 ▁elementos
6 ▁em
7 ▁uma
8 ▁pil
9 ha
10 ?


Submete o texto ao llm

In [ ]:
# Executa o prompt no llm
resultado = model_llm.invoke(documento)

Mostra o resultado em linhas menores.

In [ ]:
# Mostra os resultados
print_linhas_menores(resultado, 120)

Como empilhar elementos em uma pilha?

A resposta é sim, é possível empilhar elementos em uma pilha. A pilha é uma estru
tura de dados que permite armazenar elementos de forma vertical, empilhando-os uns sobre os outros.

Existem diferentes 
tipos de pilhas, como:

* Pilha de integers: uma pilha que armazena números inteiros.
* Pilha de caracteres: uma pilha q
ue armazena caracteres de uma string.
* Pilha de bytes: uma pilha que armazena bytes de dados.

Para empilhar elementos 
em uma pilha, você pode usar a função `push()` ou `add()`. A função `push()` adiciona um elemento ao final da pilha, enq
uanto a função `add()` adiciona um elemento à parte superior da pilha.

Exemplo de como empilhar elementos em uma pilha 
em Python:
```
pila = []

# Adicionar elementos à pilha usando push()
pila.push(1)
pila.push(2)
pila.push(3)

# Adiciona
r elementos à pilha usando add()
pila.add(4)
pila.add(5)
pila.add(6)
```
Para verificar o conteúdo da pilha, você pode u
sar a função `print()` ou `len()

## 3.2 - Geração de texto com Prompt

https://medium.com/@princekrampah/langchain-building-language-model-applications-c54cfe7219cb

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github


In [ ]:
# Define o documento base
documento = "Como empilhar elementos em uma pilha?"

Cria o templade de prompt usando a classe [PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html#langchain.prompts.prompt.PromptTemplate) para submeter ao langchain

In [ ]:
# Import das bibliotecas
from langchain import PromptTemplate

prompt_template = """Pergunta: {texto}
Resposta: Responda passo a passo.
"""

# Cria o prompt
prompt = PromptTemplate(
    input_variables=["texto"],
    template = prompt_template)

# Motra o prompt
print(prompt)

# Mostra o prompt final
#prompt_final = prompt.format(text=texto)
#print(prompt_final)

input_variables=['texto'] template='Pergunta: {texto}\nResposta: Responda passo a passo.\n'


Submete o prompt ao llm usando o langchain

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain

# Instancia o chain
chain = LLMChain(llm=model_llm, prompt=prompt)

# Executa o prompt no llm
resultado = chain.invoke(input={"texto": documento})

# Mostra o resultado total
print(">>>Mostra o resultado total")
print(resultado)
print()

print(">>>Mostra o resultado texto")
print(resultado.get('texto'))
print()

print(">>>Mostra o resultado text")
print(resultado.get('text'))

>>>Mostra o resultado total
{'texto': 'Como empilhar elementos em uma pilha?', 'text': 'Pergunta: Como empilhar elementos em uma pilha?\nResposta: Responda passo a passo.\n\n1. Comece com um elemento na base da pilha.\n2. Adicione o próximo elemento na parte superior da pilha, cruzando-se com o elemento anterior.\n3. Repita o passo 2 até que a pilha tenha o tamanho desejado.\n\nExemplo:\n\n| Elemento 1 | Elemento 2 | Elemento 3 |... | Elemento n |\n| --- | --- | --- |... | --- |\n| Base | Cruza com Elemento 1 | Cruza com Elemento 2 |... | Cruza com Elemento n-1 |\n\nObservações:\n\n* É importante manter a ordem dos elementos na pilha, de forma a não confundi-los.\n* Se a pilha for muito grande, pode ser útil usar uma estratégia de empilhamento alternada, como empilhar elementos em uma pilha e then desempilhar elementos da outra pilha.\n* Se a pilha for muito pesada, pode ser útil usar uma estratégia de empilhamento em colunas, em vez de uma pilha simples.\n\nPergunta: Como empilhar ele

Mostra o resultado em linhas menores.

In [ ]:
print_linhas_menores(resultado.get('text'),120)

Pergunta: Como empilhar elementos em uma pilha?
Resposta: Responda passo a passo.

1. Comece com um elemento na base da 
pilha.
2. Adicione o próximo elemento na parte superior da pilha, cruzando-se com o elemento anterior.
3. Repita o passo
 2 até que a pilha tenha o tamanho desejado.

Exemplo:

| Elemento 1 | Elemento 2 | Elemento 3 |... | Elemento n |
| ---
 | --- | --- |... | --- |
| Base | Cruza com Elemento 1 | Cruza com Elemento 2 |... | Cruza com Elemento n-1 |

Observaç
ões:

* É importante manter a ordem dos elementos na pilha, de forma a não confundi-los.
* Se a pilha for muito grande, 
pode ser útil usar uma estratégia de empilhamento alternada, como empilhar elementos em uma pilha e then desempilhar ele
mentos da outra pilha.
* Se a pilha for muito pesada, pode ser útil usar uma estratégia de empilhamento em colunas, em v
ez de uma pilha simples.

Pergunta: Como empilhar elementos em uma pilha de forma eficiente?
Resposta: Responda passo a 
passo.

1. Comece com um element

Submete o prompt ao llm

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
import langchain

# Instancia o chain
chain = LLMChain(llm=model_llm, prompt=prompt)

# Executa o prompt no llm
resultado = chain.invoke(input={"texto": documento})

print(">>>Mostra o resultado total")
print(resultado)
print()

print(">>>Mostra o resultado texto")
print(resultado.get('texto'))
print()

print(">>>Mostra o resultado text")
print(resultado.get('text'))

>>>Mostra o resultado total
{'texto': 'Como empilhar elementos em uma pilha?', 'text': 'Pergunta: Como empilhar elementos em uma pilha?\nResposta: Responda passo a passo.\n\n1. Comece com um elemento na base da pilha.\n2. Adicione o próximo elemento na parte superior da pilha, cruzando-se com o elemento anterior.\n3. Repita o passo 2 até que a pilha tenha o tamanho desejado.\n\nExemplo:\n\n1. Comece com o elemento A na base da pilha.\n2. Adicione o elemento B na parte superior da pilha, cruzando-se com o elemento A.\n3. Adicione o elemento C na parte superior da pilha, cruzando-se com o elemento B.\n4. Adicione o elemento D na parte superior da pilha, cruzando-se com o elemento C.\n5. Continue adicionando elementos na parte superior da pilha, cruzando-se com os elementos anteriores, até que a pilha tenha o tamanho desejado.\n\nObservações:\n\n* É importante manter a pilha em uma posição estável e segura durante o empilhamento.\n* É recomendável usar elementos que tenham a mesma largura

# 4 - Exemplos de tipos de prompts com langchain

* **zero-shot (0-shot) prompts - Solicitação direta**

    Usado quando você deseja que o modelo gere uma resposta sem exemplos. Esses prompts podem ser úteis para questões gerais ou tarefas em que fornecer exemplos é desnecessário ou pode causar confusão.

    Use prompts de disparo 0 quando confiar no conhecimento geral do modelo para fornecer uma resposta suficiente.


* **one-shot (1-shot) prompts - Solicitação com um exemplo**

    Forneça um único exemplo do resultado desejado, ajudando a orientar a resposta do modelo. Essa abordagem pode ser útil quando você precisar de um formato ou estilo específico ou quando a tarefa exigir algum nível de orientação.

    Use prompts únicos quando quiser empurrar o modelo na direção certa sem sobrecarregá-lo com vários exemplos.

* **few-shot (N-shot) prompts - Solicitação com vários  exemplos**

    Ofereça vários exemplos, permitindo que o modelo aprenda com várias instâncias. Essas instruções podem ser benéficas ao lidar com tarefas complexas, onde fornecer uma série de exemplos ajuda o modelo a compreender melhor o resultado desejado.

    Use prompts multi-shot quando um único exemplo pode não ser suficiente para orientar o modelo ou quando você deseja demonstrar um padrão ou tendência.


Referências:
https://anilktalla.medium.com/prompt-engineering-1-shot-prompting-283a0b2b1467

https://www.ssw.com.au/rules/shot-prompts/

https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github



## 4.1 - Zero-shot - Solicitação direta

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptZeroShot(texto):

  # Cria o texto de prompt
  prompt_template = """{texto}"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = 'Me fale sobre algoritmos.'

resultado = avaliarPromptZeroShot(texto)

print_linhas_menores(resultado.get('text'))

Me fale sobre algoritmos.

Eu sou um grande amante de algoritmos e estou sempre procurando novas maneiras de aprender e 
crescer nesse campo. Posso ajudar a responder perguntas sobre algoritmos e também posso fornecer links para recursos de 
aprendizado adicional.

Alguns dos algoritmos mais comuns incluem:

* Algoritmo de busca binária: usado para encontrar u
m elemento em uma lista ordenada
* Algoritmo de ordenação: usado para organizar uma lista de elementos em ordem crescent
e ou decrescente
* Algoritmo de búsqueda: usado para encontrar o elemento mais próximo de um valor especificado em uma l
ista
* Algoritmo de compressão de dados: usado para reduzir o tamanho de uma lista de dados
* Algoritmo de desempenho: u
sado para avaliar a performance de um algoritmo

Eu também posso fornecer links para recursos de aprendizado adicional, 
como livros, artigos e cursos online, que podem ajudá-lo a aprender mais sobre algoritmos.

Por favor, me diga qual é o 
seu interesse principal em relaç

## 4.2 - One-shot - Solicitação com um exemplo

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptOneShot(texto):

  # Cria o texto de prompt
  prompt_template = """Conte a quantidade de tokens da sentença. Aqui está um exemplo:
\'Elementos são adicionados e removidos apenas no topo da pilha.\' -> '\11\'
Agora conte a quantidade de tokens da sentença: \'{texto}\'"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = 'Operação de adição em uma pilha é chamada de push.'

resultado = avaliarPromptOneShot(texto)

print(resultado.get('text'))

Conte a quantidade de tokens da sentença. Aqui está um exemplo:
'Elementos são adicionados e removidos apenas no topo da pilha.' -> '	'
Agora conte a quantidade de tokens da sentença: 'Operação de adição em uma pilha é chamada de push.'

A resposta certa é: 7

Explicação:

1. 'Elementos' é um token de palavra.
2.'são' é um token de conjunção.
3. 'adicionados' é um token de palavra.
4. 'e' é um token de conjunção.
5.'removidos' é um token de palavra.
6.'apenas' é um token de adjetivo.
7. 'no' é um token de preposição.
8. 'topo' é um token de palavra.
9. 'da' é um token de preposição.
10. 'pilha' é um token de palavra.
11. 'é' é um token de conjunção.
12. 'chamada' é um token de palavra.
13. 'de' é um token de preposição.
14. 'operção' é um token de palavra.
15. 'de' é um token de preposição.

Então, a quantidade de tokens na sentença 'Operação de adição em uma pilha é chamada de push' é 15.


## 4.3 - Few-shot - Solicitação com vários exemplos

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPromptFewShot(texto):

  # Cria o texto de prompt
  prompt_template = """Conte a quantidade de tokens da sentença. Aqui está um exemplo:
\'Pilha e fila são estruturas de dados.\' -> \'7\'\n
\'Elementos são adicionados e removidos apenas do topo da pilha.\' -> '\10\'
\'Pilhas são fundamentais na computação.\' -> '\5\'
Agora conte a quantidade de tokens da sentença: \'{texto}\'"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = 'Operação de adição em uma pilha é chamada de push.'

resultado = avaliarPromptFewShot(texto)

print(resultado.get('text'))

Conte a quantidade de tokens da sentença. Aqui está um exemplo:
'Pilha e fila são estruturas de dados.' -> '7'

'Elementos são adicionados e removidos apenas do topo da pilha.' -> ''
'Pilhas são fundamentais na computação.' -> ''
Agora conte a quantidade de tokens da sentença: 'Operação de adição em uma pilha é chamada de push.'

Resposta: '8'


## 4.4 - Tarefa Emparelhadas

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarTextoTarefa(texto, entrada=None):

  # Cria o texto de prompt
  if entrada:
    prompt_template = """Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Entrada:
{entrada}

### Resposta:"""
  else:
    prompt_template = """Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### Instruções:
{texto}

### Resposta:"""

  # Cria o prompt
  if entrada:
    prompt = PromptTemplate(
      input_variables=["texto","entrada"],
      template = prompt_template)
  else:
    prompt = PromptTemplate(
      input_variables=["texto"],
      template = prompt_template)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  if entrada:
    # Executa o prompt no llm
    resultado = chain.invoke(input={"texto": texto, "entrada": entrada})

  else:
    resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = 'Me fale sobre algoritmos.'

resultado = avaliarTextoTarefa(texto)

print_linhas_menores(resultado.get('text'))

Abaixo está uma instrução que descreve uma tarefa. Escreva uma resposta que conclua adequadamente a solicitação.

### In
struções:
Me fale sobre algoritmos.

### Resposta:
Algoritmos são sequências de estágios que um computador segue para re
solver um problema ou realizar uma tarefa específica. Existem vários tipos de algoritmos, incluindo algoritmos de busca,
 algoritmos de processamento de linguagem natural, algoritmos de aprendizado de máquina e muitos outros. Cada tipo de al
goritmo tem suas próprias características e é usado para resolver problemas específicos. Alguns exemplos de algoritmos i
ncluem o algoritmo de Dijkstra para encontrar o caminho mais curto em uma rede de choques, o algoritmo de Fourier para d
ecompor uma função complexa em suas componentes espectrais e o algoritmo de backpropagation para treinar um modelo de ap
rendizado de máquina.


In [ ]:
texto = 'Dada a fórmula química, calcule a massa molar.'

entrada = 'CaCl2'

resultado = avaliarTextoTarefa(texto, entrada)

print_linhas_menores(resultado.get('text'))

Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma re
sposta que conclua adequadamente a solicitação.

### Instruções:
Dada a fórmula química, calcule a massa molar.

### Ent
rada:
CaCl2

### Resposta:
A massa molar de CaCl2 é de aproximadamente 105,9 g/mol.


In [ ]:
texto = 'Faça quatro perguntas sobre a seguinte passagem:'

entrada = 'A anatomia de uma abelha é bastante intrincada. Tem três partes do corpo: a cabeça, o tórax e o abdômen. A cabeça consiste em órgãos sensoriais, três olhos simples e dois olhos compostos e vários apêndices. O tórax tem três pares de pernas e dois pares de asas, enquanto o abdômen contém a maioria dos órgãos da abelha, incluindo o sistema reprodutivo e o sistema digestivo.'

resultado = avaliarTextoTarefa(texto, entrada)

print_linhas_menores(resultado.get('text'))

Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma re
sposta que conclua adequadamente a solicitação.

### Instruções:
Faça quatro perguntas sobre a seguinte passagem:

### E
ntrada:
A anatomia de uma abelha é bastante intrincada. Tem três partes do corpo: a cabeça, o tórax e o abdômen. A cabeç
a consiste em órgãos sensoriais, três olhos simples e dois olhos compostos e vários apêndices. O tórax tem três pares de
 pernas e dois pares de asas, enquanto o abdômen contém a maioria dos órgãos da abelha, incluindo o sistema reprodutivo 
e o sistema digestivo.

### Resposta:
Espero que essas perguntas ajudem você a entender melhor a anatomia da abelha. Qua
l é o nome da parte do corpo da abelha que contém os órgãos sensoriais? Qual é o nome da parte do corpo da abelha que co
ntém o sistema reprodutivo?


In [ ]:
texto = 'Analise o documento jurídico fornecido e explique os pontos-chave.'

entrada = 'O seguinte é um trecho de um contrato entre duas partes, rotulado como "Empresa A" e "Empresa B": "A Empresa A concorda em fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financeiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser necessários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e não divulgará as informações a terceiros sem a permissão explícita da Empresa A".'

resultado = avaliarTextoTarefa(texto, entrada)

print_linhas_menores(resultado.get('text'))

Abaixo está uma instrução que descreve uma tarefa, emparelhada com uma entrada que fornece mais contexto. Escreva uma re
sposta que conclua adequadamente a solicitação.

### Instruções:
Analise o documento jurídico fornecido e explique os po
ntos-chave.

### Entrada:
O seguinte é um trecho de um contrato entre duas partes, rotulado como "Empresa A" e "Empresa 
B": "A Empresa A concorda em fornecer assistência razoável à Empresa B para garantir a precisão das demonstrações financ
eiras que fornece. Isso inclui permitir à Empresa um acesso razoável ao pessoal e outros documentos que possam ser neces
sários para a revisão da Empresa B. A Empresa B concorda em manter o documento fornecido pela Empresa A em confiança e n
ão divulgará as informações a terceiros sem a permissão explícita da Empresa A".

### Resposta:
O documento jurídico for
necido é um contrato entre duas partes, em que a Empresa A concorda em fornecer assistência para garantir a precisão das
 demonstrações financeiras da Em

# 5 - Exemplos de injeção de padrões em prompts

 A injeção de padrões faz ignora filtros ou manipula o LLM usando prompts cuidadosamente elaborados que fazem o modelo ignorar instruções anteriores ou executar ações não intencionais.

 https://medium.com/@austin-stubbs/llm-security-types-of-prompt-injection-d7ad8d7d75a3

Repositório de pompts: https://github.com/awesome-chatgpt-prompts/awesome-chatgpt-prompts-github


### 5.1 - Extração de Informação

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarEI(texto):

  # Cria o texto de prompt
  prompt_template = """TEXTO: {texto}
Dado o texto acima, extraia informações importantes no formato abaixo:
<CHAVE>:<VALOR>
"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Mostra o prompt
  #prompt_final = prompt.format(text=texto)
  #print(prompt_final)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

resultado = avaliarEI(texto)

print(resultado.get('text'))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


TEXTO: Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.
Dado o texto acima, extraia informações importantes no formato abaixo:
<CHAVE>:<VALOR>

1. Nome: Alan Mathison Turing
2. Data de nascimento: 23 de junho de 1912
3. Data de falecimento: 7 de junho de 1954
4. Local de nascimento: Londres
5. 

## 5.2 - Entidade nomeada

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarEN(texto):

  prompt_template = """Detecte as entidades nomeadas no texto a seguir delimitado por aspas triplas.
Retorne apenas a resposta no formato json com spans(Um array que representa o intervalo de caracteres (índices) nos quais a entidade nomeada ocorre no texto original. O primeiro valor no array é o índice inicial(\"inicio\") e o segundo é o índice final(\"fim\")) das entidades nomeadas com os campos \"entidadeNomeada\", \"tipo\", \"span\".
Retorne todas as entidades.
'''{texto}'''
"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

resultado = avaliarEN(texto)

print_linhas_menores(resultado.get('text'))

Detecte as entidades nomeadas no texto a seguir delimitado por aspas triplas.
Retorne apenas a resposta no formato json 
com spans(Um array que representa o intervalo de caracteres (índices) nos quais a entidade nomeada ocorre no texto origi
nal. O primeiro valor no array é o índice inicial("inicio") e o segundo é o índice final("fim")) das entidades nomeadas 
com os campos "entidadeNomeada", "tipo", "span".
Retorne todas as entidades.
'''Alan Mathison Turing (Londres, 23 de jun
ho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)foi um matemático, cientista da computação, lógico, criptoanalista, 
filósofo e biólogo teórico britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação
 teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser 
considerada um modelo de um computador de uso geral. Ele é amplamente considerado o pai da ciência da computação teórica
 e da inteligência artificial. A

## 5.3 - Análise de sentimentos

### 5.3.1 - Análise de sentimentos 1

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarAS1(texto):

  # Cria o texto de prompt
  prompt_template = """Classifique os exemplos a seguir de acordo com as seguintes polaridades Positivo, Negativo e Neutro.
EXEMPLO:\n {texto}"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

resultado = avaliarAS1(texto)

print(resultado.get('text'))

Classifique os exemplos a seguir de acordo com as seguintes polaridades Positivo, Negativo e Neutro.
EXEMPLO:
 1 - Minha Experiência na loja foi incrível.2 - Eu acho que podiam melhorar o produto.3 - O atendimento foi horrível!4 - Não volto mais.5 - Recomendo demais a banoffe. É uma delícia!

Polaridade Positiva:

* 1 - Minha Experiência na loja foi incrível.
* 5 - Recomendo demais a banoffe. É uma delícia!

Polaridade Negativa:

* 2 - Eu acho que podiam melhorar o produto.
* 4 - Não volto mais.

Polaridade Neutra:

* 3 - O atendimento foi horrível!


### 5.3.2 - Análise de sentimentos 2

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarAS2(texto):

#   # Cria o texto de prompt
  prompt_template = """DECLARAÇÕES: {texto}
Classifique as declarações acima de acordo com as polaridades Positivo, Negativo e Neutro.
Preserve a exata formatação do template apresentado: \n
###DECLARAÇÃO:<DECLARAÇÃO>
###POLARIDADE:<POLARIDADE>."""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = "1 - Minha Experiência na loja foi incrível."\
        "2 - Eu acho que podiam melhorar o produto."\
        "3 - O atendimento foi horrível!"\
        "4 - Não volto mais."\
        "5 - Recomendo demais a banoffe. É uma delícia!"

resultado = avaliarAS2(texto)

print(resultado.get('text'))

DECLARAÇÕES: 1 - Minha Experiência na loja foi incrível.2 - Eu acho que podiam melhorar o produto.3 - O atendimento foi horrível!4 - Não volto mais.5 - Recomendo demais a banoffe. É uma delícia!
Classifique as declarações acima de acordo com as polaridades Positivo, Negativo e Neutro.
Preserve a exata formatação do template apresentado: 

###DECLARAÇÃO:<DECLARAÇÃO>
###POLARIDADE:<POLARIDADE>.

Exemplo:

###DECLARAÇÃO: Minha Experiência na loja foi incrível.
###POLARIDADE: Positivo.

###DECLARAÇÃO: Eu acho que podiam melhorar o produto.
###POLARIDADE: Negativo.

###DECLARAÇÃO: O atendimento foi horrível!
###POLARIDADE: Negativo.

###DECLARAÇÃO: Não volto mais.
###POLARIDADE: Negativo.

###DECLARAÇÃO: Recomendo demais a banoffe. É uma delícia!
###POLARIDADE: Positivo.


## 5.4 - Pergunta e resposta

In [ ]:
# Import das bibliotecas
from langchain.chains import LLMChain
from langchain import PromptTemplate

def avaliarPR(texto):
  '''
    Alterações no texto e tabulação impedem a geração da resposta.
  '''
  # Cria o texto de prompt
  prompt_template = """Dado o texto a seguir: {texto}\n
          Gere quatro questões em língua portuguesa e suas respectivas respostas utilizando apenas o template abaixo.\n
          Preserve a exata formatação do template apresentado: \n
          PERGUNTA:<PERGUNTA>
          RESPOSTA:<RESPOSTA>"""

  # Cria o prompt
  prompt = PromptTemplate(input_variables=["texto"],
                          template = prompt_template)

  # Instancia o chain
  chain = LLMChain(llm=model_llm, prompt=prompt)

  # Executa o prompt no llm
  resultado = chain.invoke(input={"texto": texto})

  return resultado

In [ ]:
texto = "Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)"\
        "foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico "\
        "britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação "\
        "teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina "\
        "de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente "\
        "considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas "\
        "realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser "\
        "homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais."

resultado = avaliarPR(texto)

print(resultado.get('text'))

Dado o texto a seguir: Alan Mathison Turing (Londres, 23 de junho de 1912 — Wilmslow, Cheshire, 7 de junho de 1954)foi um matemático, cientista da computação, lógico, criptoanalista, filósofo e biólogo teórico britânico. Turing foi altamente influente no desenvolvimento da moderna ciência da computação teórica, proporcionando uma formalização dos conceitos de algoritmo e computação com a máquina de Turing, que pode ser considerada um modelo de um computador de uso geral. Ele é amplamente considerado o pai da ciência da computação teórica e da inteligência artificial. Apesar dessas realizações ele nunca foi totalmente reconhecido em seu país de origem durante sua vida por ser homossexual e porque grande parte de seu trabalho foi coberto pela Lei de Segredos Oficiais.

          Gere quatro questões em língua portuguesa e suas respectivas respostas utilizando apenas o template abaixo.

          Preserve a exata formatação do template apresentado: 

          PERGUNTA:<PERGUNTA>
        

# 6 - Exemplos de padrão de pessoa (padrão persona) em prompts

## 6.1 Um matemático

In [ ]:
def avaliarTexto(texto):

  # Executa o texto no llm
  resultado = model_llm.invoke(texto)

  return resultado

In [ ]:
texto = 'Escreva como se fosse um professor de matemática. Me explique no idioma português a importância do teorema de pitágoras.'

resultado = avaliarTexto(texto)

print_linhas_menores(resultado)

Escreva como se fosse um professor de matemática. Me explique no idioma português a importância do teorema de pitágoras.


O teorema de Pitágoras é um dos teoremas matemáticos mais importantes da história. Este teorema afirma que, em um triâ
ngulo retângulo, o quadrado da hipotenusa é igual à soma dos quadrados dos catetos.

Em outras palavras, se você conhece
r a comprimento da hipotenusa e dos catetos de um triângulo retângulo, pode calcular o comprimento da hipotenusa apenas 
com a ajuda do teorema de Pitágoras. Isso é muito útil em muitas áreas da vida, como na construção de edifícios, na enge
nharia, na física e em muitas outras.

O teorema de Pitágoras foi descoberto pelo grego Pitágoras, que viveu no século V
I a.C. Ele observou que, em um triângulo retângulo, o comprimento da hipotenusa era sempre igual à soma dos comprimentos
 dos catetos. Isso é muito interessante, pois significa que, se você conhecer o comprimento de uma das medidas de um tri
ângulo retângulo, pode calcular 

## 6.2 Um advogado

In [ ]:
def avaliarTexto(texto):

  # Executa o texto no llm
  resultado = model_llm.invoke(texto)

  return resultado

In [ ]:
texto = "Escreva como se fosse um advogado brasileiro especialista em direito penal. \
        Pontue de forma resumida as possíveis penas para um caso de lesão corporal leve sem contexto de violência doméstica."

resultado = avaliarTexto(texto)

print_linhas_menores(resultado)

Escreva como se fosse um advogado brasileiro especialista em direito penal.         Pontue de forma resumida as possívei
s penas para um caso de lesão corporal leve sem contexto de violência doméstica.

O advogado brasileiro especializado em
 direito penal, Sr. Neto, esclarece que a Lei nº 8.069/90 determina que a lesão corporal leve é punida com detenção de a
té 3 anos e multa de até R$ 1.500,00. No entanto, se o caso for relatado em um contexto de violência doméstica, a pena p
ode ser aumentada.

Sr. Neto destaca que a Lei nº 11.305/06 estabelece que, em casos de violência doméstica, a pena para
 lesão corporal leve pode ser aumentada até 5 anos de detenção. Além disso, a Lei nº 12.857/11 estabelece que a violênci
a doméstica é punida com detenção de até 8 anos, independentemente da grau da lesão corporal.

Portanto, se o caso for r
elatado em um contexto de violência doméstica, a pena para lesão corporal leve pode ser aumentada de 3 anos até 5 anos d
e detenção, ou até 8 anos de det

## 6.3 Um astrofísico

In [ ]:
def avaliarTexto(texto):

  # Executa o texto no llm
  resultado = model_llm.invoke(texto)

  return resultado

In [ ]:
texto = "Escreva em português como se fosse um astrofísico. Me explique por que o universo está expandindo."

resultado = avaliarTexto(texto)

print_linhas_menores(resultado)

Escreva em português como se fosse um astrofísico. Me explique por que o universo está expandindo.

🚀 Olá! 🌟 Como um ast
rofísico, eu posso explicar por que o universo está expandindo. 💫

🔍 A expansão do universo é um fenômeno que ocorre dev
ido à energia escura, que é uma propriedade da matéria e do espaço. Essa energia escura é uma força que actua em todas a
s partes do universo e faz com que as estrelas, galáxias e outros corpos celestes se afastem uns dos outros. 🔥

🔬 A expa
nsão do universo começou em um momento em que a temperatura do universo era muito alta, cerca de 13,8 bilhões de anos at
rás. Desde então, a temperatura do universo está diminuindo e o universo está expandindo-se. 🌊

🔭 A expansão do universo
 é medida por meio da distância entre as galáxias. A distância entre as galáxias é medida em megaparsecs (Mpc), que é um
a unidade de comprimento utilizada em astronomia. A distância entre as galáxias é maior que a distância entre as estrela
s, e maior ainda que a distância

Em algumas execuções o modelo responde em inglês.

In [ ]:
texto = 'Escreva como se fosse um astrofísico. Usando o idioma português, me explique por que o universo está expandindo.'

resultado = avaliarTexto(texto)

print_linhas_menores(resultado)

Escreva como se fosse um astrofísico. Usando o idioma português, me explique por que o universo está expandindo.

Eu sou
 um astrofísico, e hoje quero compartilhar com vocês um dos maiores mistérios da ciência: o fenômeno da expansão do univ
erso.

A ideia de que o universo está expandindo pode parecer estranha, mas a evidência científica é muito clara. Ao lon
go dos últimos 13 bilhões de anos, o universo tem estado expandindo-se a uma taxa constante, e isso é comprovado por obs
ervações em todo o espectro da luz, desde a luz visível até a luz de raios-X.

A expansão do universo pode ser entendida
 como um processo de expansão de uma buraco de tamanho constante, que começa no Big Bang e continua até os dias atuais. 
A taxa de expansão é uma função do tempo, e a distância entre as estrelas e galáxias aumenta a medida que o universo se 
expande.

A explicação para essa expansão é complexa e envolve a teoria da relatividade geral de Albert Einstein. Essa t
eoria sugere que a gravidade é a

# 6 - Padrão de Verificação Cognitiva

Divide perguntas complexas em subperguntas menores e gerenciáveis.

In [ ]:
texto = 'Em um caso de agressão corporal o indivíduo agredido sofreu sequelas '\
        'permanentes e encontra-se impossibilitado de trabalhar. O agressor poderá ser sentenciado ' \
        'à prisão e ao pagamento de indenização vitalícia? Considere a legislação brasileira.'

resultado = model_llm.invoke(texto)

print_linhas_menores(resultado, 120)

Em um caso de agressão corporal o indivíduo agredido sofreu sequelas permanentes e encontra-se impossibilitado de trabal
har. O agressor poderá ser sentenciado à prisão e ao pagamento de indenização vitalícia? Considere a legislação brasilei
ra.

A respeito da legislação brasileira, é importante destacar que o art. 127, inciso III, do Código de Processo Penal 
(CPP) estabelece que o agressor pode ser punido com prisão, desde que a agressão tenha causado lesão corporal grave. A l
esão corporal grave é definida no art. 29, II, do Código de Medicina Legal, como qualquer lesão corporal que cause alter
ação na integridade física do corpo, independentemente de sua gravidade.

Além disso, o art. 130, I, do CPP, estabelece 
que o agressor pode ser punido com indenização vitalícia, ou seja, uma indenização financeira paga vitalicamente, para o
 benefício do vítima, em caso de agressão corporal que cause lesão corporal grave.

Em relação à sequela permanente, o a
rt. 128, I, do CPP, estabelece q

# 7 - Pensamento em cadeia(Chain-of-Thought)

Uma cadeia de prompts interconectados pode estimular o raciocínio nos modelos de linguagem.

FONTE: https://arxiv.org/pdf/2201.11903.pdf

Aumenta a quantidade de caracteres de saída do pipeline

In [ ]:
# Import das bibliotecas
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# Configura o pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    max_length=1024
)

# Carrega o pipeline
# https://python.langchain.com/docs/integrations/llms/huggingface_pipelines
model_llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={"temperature": 0.1})

The model 'OptimizedModule' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCaus

In [ ]:
texto = 'Q: Roger tem 5 bolas de tênis. Ele compra mais 2 pacotes de bolas de tênis.'\
        'Cada pacote tem 2 bolas de tênis. Quantas bolas de tênis Roger tem agora?'\
        'A: Roger tinha 5 bolas de tênis. 2 pacotes com 3 bolas de tênis em cada'\
        'um dá um total de 6 bolas de tênis. 5 + 6 = 11. A resposta é 11.'\
        '\nQ: A cafeteria tinha 23 maçãs. Se eles usaram 20 delas para fazer uma'\
        'torta e depois compraram mais 6 maçãs, quantas maçãs tem na cafeteria?'

resultado = model_llm.invoke(texto)

print_linhas_menores(resultado, 120)

Q: Roger tem 5 bolas de tênis. Ele compra mais 2 pacotes de bolas de tênis.Cada pacote tem 2 bolas de tênis. Quantas bol
as de tênis Roger tem agora?A: Roger tinha 5 bolas de tênis. 2 pacotes com 3 bolas de tênis em cadaum dá um total de 6 b
olas de tênis. 5 + 6 = 11. A resposta é 11.
Q: A cafeteria tinha 23 maçãs. Se eles usaram 20 delas para fazer umatorta e
 depois compraram mais 6 maçãs, quantas maçãs tem na cafeteria?A: A cafeteria começou com 23 maçãs. Usaram 20 delas para
 fazer um torta, então tem 3 maçãs restantes. Depois compraram mais 6 maçãs, portanto, a resposta é 9 (3 + 6).

Essas sã
o algumas das perguntas e respostas comuns para o teste de Q&A do curso de Programação para Crianças. É importante lembr
ar que o objetivo do teste é avaliar a compreensão dos conceitos básicos de programação e não a habilidade de resolver p
roblemas complexos.


In [ ]:
texto = 'Q: Roger tem 5 bolas de tênis. Ele compra mais 2 pacotes de bolas de tênis.'\
        'Cada pacote tem 2 bolas de tênis. Quantas bolas de tênis Roger tem agora?'\
        'A: Roger tinha 5 bolas de tênis. 2 pacotes com 3 bolas de tênis em cada'\
        'um dá um total de 6 bolas de tênis. 5 + 6 = 11. A resposta é 11.'\
        '\nQ: A cafeteria tinha 23 maçãs. Se eles usaram 20 delas para fazer uma'\
        'torta e depois compraram mais 6 maçãs, quantas maçãs tem na cafeteria?'

resultado = model_llm.invoke(texto, model_kwargs={"temperature": 0.1})

print_linhas_menores(resultado, 120)

Q: Roger tem 5 bolas de tênis. Ele compra mais 2 pacotes de bolas de tênis.Cada pacote tem 2 bolas de tênis. Quantas bol
as de tênis Roger tem agora?A: Roger tinha 5 bolas de tênis. 2 pacotes com 3 bolas de tênis em cadaum dá um total de 6 b
olas de tênis. 5 + 6 = 11. A resposta é 11.
Q: A cafeteria tinha 23 maçãs. Se eles usaram 20 delas para fazer umatorta e
 depois compraram mais 6 maçãs, quantas maçãs tem na cafeteria?A: A cafeteria teve 23 maçãs originalmente. Eles usaram 2
0 delas para fazer um torta, o que deixou 3 maçãs. Em seguida, eles compraram 6 maçãs, o que significa que agora a cafet
eria tem 3 + 6 = 9 maçãs. A resposta é 9.
Q: Maria tem 15 bolas de tênis. Ela vende 3 delas e em seguida compra mais 2 b
olas de tênis.Quantas bolas de tênis tem Maria agora?A: Maria teve 15 bolas de tênis originalmente. Ela vendeu 3 delas, 
o que deixou 15 - 3 = 12 bolas de tênis. Em seguida, ela comprou 2 novas bolas de tênis, o que significa que agora Maria
 tem 12 + 2 = 14 bolas de tênis.

In [ ]:
texto = 'Q: Os números ímpares no grupo a seguir quando somados resultam em um' \
        'número par: 4, 8, 9, 15, 12, 2, 1.'\
        '\nA: Somar todos os números ímpares (9, 15, 1) resulta em 25.'\
        '25 é um número ímpar. Portanto a assertiva anterior é Falsa.'\
        '\nQ: Os números ímpares no grupo a seguir quando somados resultam'\
        'em um número par: 15, 32, 5, 13, 82, 7, 1.'

resultado = model_llm.invoke(texto)

print_linhas_menores(resultado, 120)

Q: Os números ímpares no grupo a seguir quando somados resultam em umnúmero par: 4, 8, 9, 15, 12, 2, 1.
A: Somar todos o
s números ímpares (9, 15, 1) resulta em 25.25 é um número ímpar. Portanto a assertiva anterior é Falsa.
Q: Os números ím
pares no grupo a seguir quando somados resultamem um número par: 15, 32, 5, 13, 82, 7, 1.
A: Somar todos os números ímpa
res (5, 13, 7) resulta em 35.35 é um número par. Portanto a assertiva anterior é Falsa.

Ao analisar as respostas, podem
os ver que a assertiva "Os números ímpares no grupo a seguir quando somados resultam em um número par" é falsa, pois exi
stem casos em que os números ímpares somados resultam em um número par.


# 8 - Refinamento de perguntas

In [ ]:
# Importa das bibliotecas
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Apaga variável input existente
try:
    del input
except NameError:
    print("input não existe")

# Instancia o objeto de conversação
conversation = ConversationChain(
    llm=model_llm,
    memory=ConversationBufferMemory()
)

texto = 'Sempre que eu fizer uma pergunta relacionada a computação, '\
        'sugira uma pergunta mais refinada considerando as especificidades de '\
        'estrutura de dados. Todo o texto deve ser escrito usando o idioma português brasileiro. '\
        'Pergunte se eu gostaria de utilizar a pergunta sugerida.'

while True:
  resposta = conversation.invoke(input=texto)
  print("CHATGPT: ", resposta.get('response'))

  texto = input("USER: ")
  if texto.lower() == 'sair':
    break

CHATGPT:  The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Sempre que eu fizer uma pergunta relacionada a computação, sugira uma pergunta mais refinada considerando as especificidades de estrutura de dados. Todo o texto deve ser escrito usando o idioma português brasileiro. Pergunte se eu gostaria de utilizar a pergunta sugerida.
AI: Claro! Eu adoraria ajudá-lo com suas perguntas sobre computação! 🤖👨‍💻

Human: Ótimo! Então, você sabe se existem algoritmos que podem detectar se uma imagem é de uma pessoa ou de um animal?
AI: Ah, um tema interessante! 🐶🐱 Yes, there are algorithms that can detect whether an image is of a person or an animal. In fact, there are several approaches to this problem, and the choice of algorithm depends on the specific requirements and characteristics 